In [1]:
#프로젝트의 루트 디렉토리를 설정하기 위한 코드
import os
current_dir = os.getcwd()  # 현재 작업 디렉토리
print(os.path.dirname(current_dir))
# print(os.path.dirname(os.path.dirname(current_dir)))
# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = os.path.dirname(current_dir)
os.chdir(project_root)  # 루트 경로로 변경

c:\sb-fsts\sb-fsts


In [2]:
import datetime
import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas as pd
import numpy as np
import requests
from datetime import timedelta

In [10]:
#예시 데이터
class KisDomesticDailyChartBar:
    def __init__(self, time, open, close, high, low, volume, amount, change):
        self.time = time
        self.open = open
        self.close = close
        self.high = high
        self.low = low
        self.volume = volume
        self.amount = amount
        self.change = change


ohlc = [
    KisDomesticDailyChartBar("2023-01-01T00:00:00+09:00", 100000, 95000, 100000, 95000, 0, 0, 0),
    KisDomesticDailyChartBar("2023-01-02T00:00:00+09:00", 93000, 110000, 115000, 92000, 0, 0, 0),
    KisDomesticDailyChartBar("2023-01-03T00:00:00+09:00", 98000, 120000, 120000, 98000, 0, 0, 0),
    KisDomesticDailyChartBar("2023-01-04T00:00:00+09:00", 172500, 174000, 175000, 171000, 0, 0, 0),
]


In [8]:
from app.utils.auto_trading_bot import AutoTradingBot
from datetime import date, time

user_name = '홍석형'
auto_trading_stock = AutoTradingBot(user_name)

symbol = '006400'

start_date = date(2023, 6, 1)
end_date = date(2024, 12, 1)

auto_trading_stock._get_ohlc(symbol, start_date, end_date)


실전투자 API 객체가 성공적으로 생성되었습니다.


[KisDomesticDailyChartBar(time='2023-06-01T00:00:00+09:00', open=718000, close=708000, high=721000, low=706000, volume=134626, amount=95733104000, change=-10000),
 KisDomesticDailyChartBar(time='2023-06-02T00:00:00+09:00', open=708000, close=715000, high=718000, low=706000, volume=155491, amount=110838316000, change=7000),
 KisDomesticDailyChartBar(time='2023-06-05T00:00:00+09:00', open=722000, close=717000, high=722000, low=710000, volume=77553, amount=55515221000, change=2000),
 KisDomesticDailyChartBar(time='2023-06-07T00:00:00+09:00', open=722000, close=735000, high=740000, low=719000, volume=228145, amount=167172463566, change=18000),
 KisDomesticDailyChartBar(time='2023-06-08T00:00:00+09:00', open=736000, close=729000, high=736000, low=717000, volume=197396, amount=143625853000, change=-6000),
 KisDomesticDailyChartBar(time='2023-06-09T00:00:00+09:00', open=729000, close=735000, high=735000, low=726000, volume=150901, amount=110401874000, change=6000),
 KisDomesticDailyChartBar(t

In [9]:
ohlc = auto_trading_stock._get_ohlc(symbol, start_date, end_date)

In [5]:
def trading_signal_penetrating(ohlc):
    """
    관통형
    매수/매도 신호를 판단하는 함수.
    :param ohlc: 날짜별 OHLC 데이터 (KisDomesticDailyChartBar 객체 리스트)
    :return: 매수 성공 목록과 개별 신호
    """
    results = []  # 전체 결과 저장
    successful_trades = []  # 매수 성공 목록
    buy_signals = []  # 차트에 표시할 매수 신호
    timestamps = []  # 타임스탬프 저장
    
    trading_history = {
        'history': [],  # 매매 기록
        'average_price': 0,
        'realized_pnl': 0,
        'unrealized_pnl': 0,
        'total_cost': 0,
        'total_quantity': 0
    }

    if len(ohlc) < 3:
        raise ValueError("3일 이상의 데이터가 필요합니다.")

    for i in range(2, len(ohlc)):
        d_2 = ohlc[i - 2]
        d_1 = ohlc[i - 1]
        current = ohlc[i]

        # D-2 조건
        d_2_condition = d_2.close < d_2.open
        d_2_long_bear = abs(d_2.close - d_2.open) >= (float(d_2.open) * 0.02) # 2%하락으로 계산
        print(f"\n[D-2 조건] 날짜: {d_2.time}")
        print(f"  D-2종가 < D-2시초가: {d_2.close} < {d_2.open} = {d_2_condition}")
        print(f"  장대음봉: |{d_2.close} - {d_2.open}| >= {float(d_2.open) * 0.02} = {d_2_long_bear}")
        # D-1 조건
        d_1_condition = (
            d_1.open < d_2.low and
            d_1.close > d_2.close + (d_2.open - d_2.close) / 2
        )
        print(f"\n[D-1 조건] 날짜: {d_1.time}")
        print(f"  D-1 시초가 < D-2 최저가: {d_1.open} < {d_2.low} = {d_1.open < d_2.low}")
        
        # 모든 조건 충족 여부 확인
        all_conditions_met = d_2_condition and d_2_long_bear and d_1_condition

        if all_conditions_met:
            # 매매 시점 나누기
            buy_signal_d2 = current.close > d_2.high
            buy_signal_d1 = current.close > d_1.high
            print(f"\n[매매 시점] 날짜: {current.time}")
            print(f"  현재 종가 > D-2 최고가: {current.close} > {d_2.high} = {buy_signal_d2}")
            print(f"  현재 종가 > D-1 최고가: {current.close} > {d_1.high} = {buy_signal_d1}")
            
            if buy_signal_d2 or buy_signal_d1:
                # 매수 성공 여부
                buy_condition = "P > D-2.h" if buy_signal_d2 else "P > D-1.h"
                success = True  # 매수 성공

                # 결과 저장
                trade_result = {
                    "날짜": current.time,
                    "매수/매도": "매수신호",
                    "특징": "D-2 장대음봉",
                    "매매시점": buy_condition,
                    "손절조건": "미충족" if current.close >= d_1.low else "충족",
                    "매수 성공 여부": "성공" if success else "실패",
                }
                results.append(trade_result)
                # 매수 성공 목록에 추가
                successful_trades.append(trade_result)
                
                                # 매수 신호 차트 데이터 추가
                buy_signals.append((current.time, current.close))
            else:
                # 매수 신호는 있으나 성공하지 않음
                results.append({
                    "날짜": current.time,
                    "매수/매도": "매수신호",
                    "특징": "D-2 장대음봉",
                    "매매시점": "미충족",
                    "손절조건": "미충족",
                    "매수 성공 여부": "실패",
                })
        else:
            # 조건 충족하지 않음
            results.append({
                "날짜": current.time,
                "매수/매도": "조건 미충족",
                "특징": "미충족",
                "매매시점": "미충족",
                "손절조건": "미충족",
                "매수 성공 여부": "실패",
            })

        # 차트 데이터 기록
        timestamps.append(current.time)
        
    return results, successful_trades, buy_signals, timestamps

# 매수/매도 신호 판단
signals, successful_trades, buy_signals, timestamps = trading_signal_penetrating(ohlc)

# 전체 신호 출력
print("전체 신호:")
for signal in signals:
    print(signal)

# 매수 성공 목록 출력
print("\n매수 성공 목록:")
for trade in successful_trades:
    print(trade)



[D-2 조건] 날짜: 2020-06-01 00:00:00+09:00
  D-2종가 < D-2시초가: 1.505E+4 < 1.465E+4 = False
  장대음봉: |1.505E+4 - 1.465E+4| >= 293.0 = True

[D-1 조건] 날짜: 2020-06-02 00:00:00+09:00
  D-1 시초가 < D-2 최저가: 1.485E+4 < 1.405E+4 = False

[D-2 조건] 날짜: 2020-06-02 00:00:00+09:00
  D-2종가 < D-2시초가: 1.54E+4 < 1.485E+4 = False
  장대음봉: |1.54E+4 - 1.485E+4| >= 297.0 = True

[D-1 조건] 날짜: 2020-06-03 00:00:00+09:00
  D-1 시초가 < D-2 최저가: 1.57E+4 < 1.455E+4 = False

[D-2 조건] 날짜: 2020-06-03 00:00:00+09:00
  D-2종가 < D-2시초가: 1.515E+4 < 1.57E+4 = True
  장대음봉: |1.515E+4 - 1.57E+4| >= 314.0 = True

[D-1 조건] 날짜: 2020-06-04 00:00:00+09:00
  D-1 시초가 < D-2 최저가: 1.475E+4 < 1.505E+4 = True

[D-2 조건] 날짜: 2020-06-04 00:00:00+09:00
  D-2종가 < D-2시초가: 1.395E+4 < 1.475E+4 = True
  장대음봉: |1.395E+4 - 1.475E+4| >= 295.0 = True

[D-1 조건] 날짜: 2020-06-05 00:00:00+09:00
  D-1 시초가 < D-2 최저가: 1.38E+4 < 1.355E+4 = False

[D-2 조건] 날짜: 2020-06-05 00:00:00+09:00
  D-2종가 < D-2시초가: 1.425E+4 < 1.38E+4 = False
  장대음봉: |1.425E+4 - 1.38E+4| >= 276.0 = 

In [10]:
def trading_signal_engulfing(ohlc):
    """
    상승장악형1 매수/매도 신호를 판단하는 함수.
    :param ohlc: 날짜별 OHLC 데이터 (KisDomesticDailyChartBar 객체 리스트)
    :return: 매수 성공 목록과 개별 신호
    """
    results = []  # 전체 결과 저장
    successful_trades = []  # 매수 성공 목록

    if len(ohlc) < 3:
        raise ValueError("3일 이상의 데이터가 필요합니다.")

    for i in range(2, len(ohlc)):
        d_2 = ohlc[i - 2]  # D-2 데이터
        d_1 = ohlc[i - 1]  # D-1 데이터
        current = ohlc[i]  # 당일 데이터

        # D-2 조건 디버깅 (음봉)
        d_2_condition = d_2.close < d_2.open
        print(f"\n[D-2 조건] 날짜: {d_2.time}")
        print(f"  종가 < 시초가: {d_2.close} < {d_2.open} = {d_2_condition}")

        # D-1 조건 디버깅
        d_1_condition = (
            d_1.open < d_2.low and
            d_1.close > d_2.high
        )
        print(f"\n[D-1 조건] 날짜: {d_1.time}")
        print(f"  시초가 < D-2 최저가: {d_1.open} < {d_2.low} = {d_1.open < d_2.low}")
        print(f"  종가 > D-2 최고가: {d_1.close} > {d_2.high} = {d_1.close > d_2.high}")

        # 매매 시점 디버깅
        buy_signal = current.close > d_1.high
        print(f"\n[매매 시점] 날짜: {current.time}")
        print(f"  종가 > D-1 최고가: {current.close} > {d_1.high} = {buy_signal}")

        # 손절 조건 디버깅
        stoploss = current.close < d_1.low
        print(f"\n[손절 조건] 날짜: {current.time}")
        print(f"  종가 < D-1 최저가: {current.close} < {d_1.low} = {stoploss}")

        # 모든 조건 충족 여부 확인
        all_conditions_met = d_2_condition and d_1_condition
        print(f"\n[전체 조건 충족 여부] 날짜: {current.time}")
        print(f"  D-2 조건: {d_2_condition}")
        print(f"  D-1 조건: {d_1_condition}")
        print(f"  모든 조건 충족: {all_conditions_met}")

        if all_conditions_met:
            if buy_signal:
                trade_result = {
                    "날짜": current.time,
                    "매수/매도": "매수신호",
                    "특징": "상승장악형1",
                    "매매시점": "P > H",
                    "손절조건": "미충족" if not stoploss else "충족",
                    "매수 성공 여부": "성공",
                }
                results.append(trade_result)
                successful_trades.append(trade_result)
            else:
                results.append({
                    "날짜": current.time,
                    "매수/매도": "매수신호",
                    "특징": "상승장악형1",
                    "매매시점": "미충족",
                    "손절조건": "미충족" if not stoploss else "충족",
                    "매수 성공 여부": "실패",
                })
        else:
            results.append({
                "날짜": current.time,
                "매수/매도": "조건 미충족",
                "특징": "미충족",
                "매매시점": "미충족",
                "손절조건": "미충족",
                "매수 성공 여부": "실패",
            })

    return results, successful_trades

# 매수/매도 신호 판단
signals, successful_trades = trading_signal_engulfing(ohlc)

# 전체 신호 출력
print("\n전체 신호:")
for signal in signals:
    print(signal)

# 매수 성공 목록 출력
print("\n매수 성공 목록:")
for trade in successful_trades:
    print(trade)



[D-2 조건] 날짜: 2023-06-01 00:00:00+09:00
  종가 < 시초가: 7.08E+5 < 7.18E+5 = True

[D-1 조건] 날짜: 2023-06-02 00:00:00+09:00
  시초가 < D-2 최저가: 7.08E+5 < 7.06E+5 = False
  종가 > D-2 최고가: 7.15E+5 > 7.21E+5 = False

[매매 시점] 날짜: 2023-06-05 00:00:00+09:00
  종가 > D-1 최고가: 7.17E+5 > 7.18E+5 = False

[손절 조건] 날짜: 2023-06-05 00:00:00+09:00
  종가 < D-1 최저가: 7.17E+5 < 7.06E+5 = False

[전체 조건 충족 여부] 날짜: 2023-06-05 00:00:00+09:00
  D-2 조건: True
  D-1 조건: False
  모든 조건 충족: False

[D-2 조건] 날짜: 2023-06-02 00:00:00+09:00
  종가 < 시초가: 7.15E+5 < 7.08E+5 = False

[D-1 조건] 날짜: 2023-06-05 00:00:00+09:00
  시초가 < D-2 최저가: 7.22E+5 < 7.06E+5 = False
  종가 > D-2 최고가: 7.17E+5 > 7.18E+5 = False

[매매 시점] 날짜: 2023-06-07 00:00:00+09:00
  종가 > D-1 최고가: 7.35E+5 > 7.22E+5 = True

[손절 조건] 날짜: 2023-06-07 00:00:00+09:00
  종가 < D-1 최저가: 7.35E+5 < 7.1E+5 = False

[전체 조건 충족 여부] 날짜: 2023-06-07 00:00:00+09:00
  D-2 조건: False
  D-1 조건: False
  모든 조건 충족: False

[D-2 조건] 날짜: 2023-06-05 00:00:00+09:00
  종가 < 시초가: 7.17E+5 < 7.22E+5 = True

[D-1 조

In [6]:
import math
from app.utils.auto_trading_bot import AutoTradingBot
import time

def simulate_penetrating_trading(ohlc_data, trade_value):
    """
    관통형 매매 시뮬레이션 및 손익 계산.
    :param ohlc_data: OHLC 데이터 (KisDomesticDailyChartBar 객체 리스트)
    :param trade_value: 매매에 사용할 금액 (KRW)
    :return: 시뮬레이션 결과
    """
    # 초기화
    trading_history = {
        'history': [],  # 매매 기록
        'average_price': 0,
        'realized_pnl': 0,
        'unrealized_pnl': 0,
        'total_cost': 0,
        'total_quantity': 0
    }

    # 시뮬레이션 데이터 저장
    buy_signals = []
    sell_signals = []
    timestamps = []

    # 시뮬레이션 실행
    for i in range(2, len(ohlc_data)):
        d_2 = ohlc_data[i - 2]
        d_1 = ohlc_data[i - 1]
        current = ohlc_data[i]

        # D-2 조건: 음봉
        d_2_condition = d_2.close < d_2.open
        d_2_long_bear = abs(d_2.close - d_2.open) >= (float(d_2.open) * 0.02)

        # D-1 조건: 특정 조건 충족
        d_1_condition = (
            d_1.open < d_2.low and
            d_1.close > d_2.close + (d_2.open - d_2.close) / 2
        )

        # 매매 시점: 현재 종가 > D-2 또는 D-1 최고가
        buy_signal_d2 = current.close > d_2.high
        buy_signal_d1 = current.close > d_1.high

        all_conditions_met = d_2_condition and d_2_long_bear and d_1_condition

        if all_conditions_met and (buy_signal_d2 or buy_signal_d1):
            # 매수
            buy_price = current.close  # 매수가는 당일 시초가로 설정
            stop_loss = d_1.low  # stop loss는 매수한 날 전 날의 최저가
            # 수량 계산
            quantity = math.floor(float(trade_value) / float(buy_price))
            if quantity >= 0:
                trading_history['history'].append({
                'position': 'BUY',
                'price': buy_price,
                'quantity': quantity,
                'stop_loss': stop_loss,
            })

            # 매수 신호 기록
            buy_signals.append((current.time, buy_price))
            timestamps.append(current.time)
            print(f"매수 시점: {current.time}, 매수가: {buy_price}, 매수량: {quantity}, 손절가: {stop_loss}")

        # 매도 조건 확인 및 실행
        for trade in trading_history['history']:
            if trade['position'] == 'BUY' and trade['quantity'] > 0:  # 매수 수량이 있어야 매도 가능
                target_price = trade['price'] + 2 * (trade['price'] - trade['stop_loss'])
                if current.close <= trade['stop_loss']:
                    # 손절 매도
                    sell_quantity = trade['quantity']  # 매도 수량은 남은 매수 수량만큼
                    sell_price = float(trade['stop_loss'])
                    trading_history['history'].append({
                        'position': 'SELL',
                        'price': trade['stop_loss'],
                        'quantity': sell_quantity,
                    })
                    sell_signals.append((current.time, trade['stop_loss']))
                    print(f"손절 매도: {current.time}, 매도가: {trade['stop_loss']}, 매도량: {sell_quantity}")
                    trade['quantity'] = 0  # 남은 수량 감소
                elif current.close >= target_price:
                    # 목표 수익 매도
                    sell_quantity = trade['quantity']  # 매도 수량은 남은 매수 수량만큼
                    sell_price = float(target_price)
                    trading_history['history'].append({
                        'position': 'SELL',
                        'price': target_price,
                        'quantity': sell_quantity,
                    })
                    sell_signals.append((current.time, target_price))
                    print(f"목표 매도: {current.time}, 매도가: {target_price}, 매도량: {sell_quantity}")
                    trade['quantity'] = 0  # 남은 수량 감소

        # 손익 계산
        total_cost = 0  # 총 비용
        total_quantity = 0  # 총 수량
        realized_pnl = 0  # 실현 손익

        for trade in trading_history['history']:
            if trade['position'] == 'BUY':
                total_cost += trade['price'] * trade['quantity']
                total_quantity += trade['quantity']
            elif trade['position'] == 'SELL':
                if total_quantity == 0:
                    continue
                sell_quantity = trade['quantity']
                sell_price = trade['price']
                average_price = total_cost / total_quantity if total_quantity > 0 else 0
                realized_pnl += (sell_price - average_price) * sell_quantity
                total_quantity -= sell_quantity
                total_cost -= average_price * sell_quantity
                total_cost = max(total_cost, 0)

        # 평균 단가 및 미실현 손익
        average_price = total_cost / total_quantity if total_quantity > 0 else 0
        unrealized_pnl = (current.close - average_price) * total_quantity if total_quantity > 0 else 0

        # 손익 결과 저장
        trading_history['average_price'] = average_price
        trading_history['realized_pnl'] = realized_pnl
        trading_history['unrealized_pnl'] = unrealized_pnl
        trading_history['total_cost'] = total_cost
        trading_history['total_quantity'] = total_quantity

        # 결과 출력
        print(f"날짜: {current.time}")
        print(f"총 비용: {total_cost} KRW, 총 보유량: {total_quantity} 주")
        print(f"평균 단가: {average_price} KRW, 실현 손익: {realized_pnl} KRW")
        print(f"미실현 손익: {unrealized_pnl} KRW\n")

    return trading_history, buy_signals, sell_signals, timestamps


In [7]:
# 시뮬레이션 실행
trade_value = 1000000  # 매매 금액
trading_history, buy_signals, sell_signals, timestamps = simulate_penetrating_trading(ohlc, trade_value)

# 최종 결과 출력
print("\n[최종 매매 기록]")
for trade in trading_history['history']:
    print(trade)


날짜: 2020-06-03 00:00:00+09:00
총 비용: 0 KRW, 총 보유량: 0 주
평균 단가: 0 KRW, 실현 손익: 0 KRW
미실현 손익: 0 KRW

날짜: 2020-06-04 00:00:00+09:00
총 비용: 0 KRW, 총 보유량: 0 주
평균 단가: 0 KRW, 실현 손익: 0 KRW
미실현 손익: 0 KRW

날짜: 2020-06-05 00:00:00+09:00
총 비용: 0 KRW, 총 보유량: 0 주
평균 단가: 0 KRW, 실현 손익: 0 KRW
미실현 손익: 0 KRW

날짜: 2020-06-08 00:00:00+09:00
총 비용: 0 KRW, 총 보유량: 0 주
평균 단가: 0 KRW, 실현 손익: 0 KRW
미실현 손익: 0 KRW

날짜: 2020-06-09 00:00:00+09:00
총 비용: 0 KRW, 총 보유량: 0 주
평균 단가: 0 KRW, 실현 손익: 0 KRW
미실현 손익: 0 KRW

날짜: 2020-06-10 00:00:00+09:00
총 비용: 0 KRW, 총 보유량: 0 주
평균 단가: 0 KRW, 실현 손익: 0 KRW
미실현 손익: 0 KRW

날짜: 2020-06-11 00:00:00+09:00
총 비용: 0 KRW, 총 보유량: 0 주
평균 단가: 0 KRW, 실현 손익: 0 KRW
미실현 손익: 0 KRW

날짜: 2020-06-12 00:00:00+09:00
총 비용: 0 KRW, 총 보유량: 0 주
평균 단가: 0 KRW, 실현 손익: 0 KRW
미실현 손익: 0 KRW

날짜: 2020-06-15 00:00:00+09:00
총 비용: 0 KRW, 총 보유량: 0 주
평균 단가: 0 KRW, 실현 손익: 0 KRW
미실현 손익: 0 KRW

날짜: 2020-06-16 00:00:00+09:00
총 비용: 0 KRW, 총 보유량: 0 주
평균 단가: 0 KRW, 실현 손익: 0 KRW
미실현 손익: 0 KRW

날짜: 2020-06-17 00:00:00+09:00
총 비용: 0 KR

In [19]:
def engulfing_logic2(ohlc):
        """
        상승장악형2 매매 로직.
        :return: 매수 성공 목록과 개별 신호
        """
        results = []
        successful_trades = []

        if len(ohlc) < 2:
            raise ValueError("2일 이상의 데이터가 필요합니다.")

        for i in range(1, len(ohlc)):
            d_1 = ohlc[i - 1]  # D-1 데이터
            current = ohlc[i]  # 당일 데이터

            # D-1 조건: D-1 종가 < D-1 시초가 (음봉)
            d_1_condition = d_1.close < d_1.open
            print(f"\n[D-1 조건] 날짜: {d_1.time}")
            print(f"  종가 < 시초가: {d_1.close} < {d_1.open} = {d_1_condition}")

            # 매매 시점 조건
            buy_signal = current.open < d_1.low and current.close > d_1.high
            print(f"\n[매매 시점] 날짜: {current.time}")
            print(f"  당일 시가 < D-1 최저가: {current.open} < {d_1.low} = {current.open < d_1.low}")
            print(f"  당일 종가 > D-1 최고가: {current.close} > {d_1.high} = {current.close > d_1.high}")

            # 손절 조건
            stoploss = current.close < d_1.low
            print(f"\n[손절 조건] 날짜: {current.time}")
            print(f"  종가 < D-1 최저가: {current.close} < {d_1.low} = {stoploss}")

            # 모든 조건 충족 여부 확인
            all_conditions_met = d_1_condition
            print(f"\n[전체 조건 충족 여부] 날짜: {current.time}")
            print(f"  D-1 조건: {d_1_condition}")
            print(f"  모든 조건 충족: {all_conditions_met}")

            if all_conditions_met:
                if buy_signal:
                    trade_result = {
                        "날짜": current.time,
                        "매수/매도": "매수신호",
                        "특징": "상승장악형2",
                        "매매시점": "SS < L and P > H",
                        "손절조건": "미충족" if not stoploss else "충족",
                        "매수 성공 여부": "성공",
                    }
                    results.append(trade_result)
                    successful_trades.append(trade_result)
                else:
                    results.append({
                        "날짜": current.time,
                        "매수/매도": "매수신호",
                        "특징": "상승장악형2",
                        "매매시점": "미충족",
                        "손절조건": "미충족" if not stoploss else "충족",
                        "매수 성공 여부": "실패",
                    })
            else:
                results.append({
                    "날짜": current.time,
                    "매수/매도": "조건 미충족",
                    "특징": "미충족",
                    "매매시점": "미충족",
                    "손절조건": "미충족",
                    "매수 성공 여부": "실패",
                })

        return results, successful_trades
    
    # 매수/매도 신호 판단
signals, successful_trades = engulfing_logic2(ohlc)

# 전체 신호 출력
print("\n전체 신호:")
for signal in signals:
    print(signal)

# 매수 성공 목록 출력
print("\n매수 성공 목록:")
for trade in successful_trades:
    print(trade)


[D-1 조건] 날짜: 2020-06-01 00:00:00+09:00
  종가 < 시초가: 1.505E+4 < 1.465E+4 = False

[매매 시점] 날짜: 2020-06-02 00:00:00+09:00
  당일 시가 < D-1 최저가: 1.485E+4 < 1.405E+4 = False
  당일 종가 > D-1 최고가: 1.54E+4 > 1.55E+4 = False

[손절 조건] 날짜: 2020-06-02 00:00:00+09:00
  종가 < D-1 최저가: 1.54E+4 < 1.405E+4 = False

[전체 조건 충족 여부] 날짜: 2020-06-02 00:00:00+09:00
  D-1 조건: False
  모든 조건 충족: False

[D-1 조건] 날짜: 2020-06-02 00:00:00+09:00
  종가 < 시초가: 1.54E+4 < 1.485E+4 = False

[매매 시점] 날짜: 2020-06-03 00:00:00+09:00
  당일 시가 < D-1 최저가: 1.57E+4 < 1.455E+4 = False
  당일 종가 > D-1 최고가: 1.515E+4 > 1.545E+4 = False

[손절 조건] 날짜: 2020-06-03 00:00:00+09:00
  종가 < D-1 최저가: 1.515E+4 < 1.455E+4 = False

[전체 조건 충족 여부] 날짜: 2020-06-03 00:00:00+09:00
  D-1 조건: False
  모든 조건 충족: False

[D-1 조건] 날짜: 2020-06-03 00:00:00+09:00
  종가 < 시초가: 1.515E+4 < 1.57E+4 = True

[매매 시점] 날짜: 2020-06-04 00:00:00+09:00
  당일 시가 < D-1 최저가: 1.475E+4 < 1.505E+4 = True
  당일 종가 > D-1 최고가: 1.395E+4 > 1.625E+4 = False

[손절 조건] 날짜: 2020-06-04 00:00:00+09:00
  종가 <

In [15]:
def counterattack_logic(ohlc):
        """
        상승 반격형 매매 로직.
        :return: 매수 성공 목록과 개별 신호
        """
        results = []
        successful_trades = []

        if len(ohlc) < 3:
            raise ValueError("3일 이상의 데이터가 필요합니다.")

        for i in range(2, len(ohlc)):
            d_2 = ohlc[i - 2]  # D-2 데이터
            d_1 = ohlc[i - 1]  # D-1 데이터
            current = ohlc[i]  # 당일 데이터

            # D-2 조건
            d_2_condition = d_2.close < d_2.open

            # D-1 조건
            midpoint = d_2.close + (d_2.open - d_2.close) / 2
            d_1_condition = (
                d_1.open < d_2.low and
                d_2.close <= d_1.close <= midpoint
            )

            # 매매 시점
            buy_signal = current.close > d_2.high

            # 손절 조건
            stoploss = current.close < d_2.low

            # 모든 조건 충족 여부 확인
            all_conditions_met = d_2_condition and d_1_condition

            if all_conditions_met:
                if buy_signal:
                    trade_result = {
                        "날짜": current.time,
                        "매수/매도": "매수신호",
                        "특징": "상승 반격형",
                        "매매시점": "P > h",
                        "손절조건": "미충족" if not stoploss else "충족",
                        "매수 성공 여부": "성공",
                    }
                    results.append(trade_result)
                    successful_trades.append(trade_result)
                else:
                    results.append({
                        "날짜": current.time,
                        "매수/매도": "매수신호",
                        "특징": "상승 반격형",
                        "매매시점": "미충족",
                        "손절조건": "미충족" if not stoploss else "충족",
                        "매수 성공 여부": "실패",
                    })
            else:
                results.append({
                    "날짜": current.time,
                    "매수/매도": "조건 미충족",
                    "특징": "미충족",
                    "매매시점": "미충족",
                    "손절조건": "미충족",
                    "매수 성공 여부": "실패",
                })

        return results, successful_trades
    
        # 매수/매도 신호 판단
signals, successful_trades = counterattack_logic(ohlc)

# 전체 신호 출력
print("\n전체 신호:")
for signal in signals:
    print(signal)

# 매수 성공 목록 출력
print("\n매수 성공 목록:")
for trade in successful_trades:
    print(trade)
    
    


전체 신호:
{'날짜': datetime.datetime(2020, 6, 3, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), '매수/매도': '조건 미충족', '특징': '미충족', '매매시점': '미충족', '손절조건': '미충족', '매수 성공 여부': '실패'}
{'날짜': datetime.datetime(2020, 6, 4, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), '매수/매도': '조건 미충족', '특징': '미충족', '매매시점': '미충족', '손절조건': '미충족', '매수 성공 여부': '실패'}
{'날짜': datetime.datetime(2020, 6, 5, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), '매수/매도': '조건 미충족', '특징': '미충족', '매매시점': '미충족', '손절조건': '미충족', '매수 성공 여부': '실패'}
{'날짜': datetime.datetime(2020, 6, 8, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), '매수/매도': '조건 미충족', '특징': '미충족', '매매시점': '미충족', '손절조건': '미충족', '매수 성공 여부': '실패'}
{'날짜': datetime.datetime(2020, 6, 9, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), '매수/매도': '조건 미충족', '특징': '미충족', '매매시점': '미충족', '손절조건': '미충족', '매수 성공 여부': '실패'}
{'날짜': datetime.datetime(2020, 6, 10, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), '매수/매도': '조건 미충족', '특징': '미충족', '매매시점': '미충족', '손절조건': '미충족', '

In [ ]:
def harami_logic(ohlc):
        """
        상승 잉태형 매매 로직.
        :return: 매수 성공 목록과 개별 신호
        """
        results = []
        successful_trades = []

        if len(ohlc) < 3:
            raise ValueError("3일 이상의 데이터가 필요합니다.")

        for i in range(2, len(ohlc)):
            d_2 = ohlc[i - 2]  # D-2 데이터
            d_1 = ohlc[i - 1]  # D-1 데이터
            current = ohlc[i]  # 당일 데이터

            # D-2 조건
            d_2_condition = d_2.close < d_2.open

            # D-1 조건
            d_1_condition = (
                d_1.close > d_2.close >= d_1.open and
                d_1.high < d_2.open and
                d_1.low > d_2.close
            )

            # 매매 시점
            buy_signal = current.close > d_2.high

            # 손절 조건
            stoploss = current.close < d_2.low

            # 모든 조건 충족 여부 확인
            all_conditions_met = d_2_condition and d_1_condition

            if all_conditions_met:
                if buy_signal:
                    trade_result = {
                        "날짜": current.time,
                        "매수/매도": "매수신호",
                        "특징": "상승 잉태형",
                        "매매시점": "P > h",
                        "손절조건": "미충족" if not stoploss else "충족",
                        "매수 성공 여부": "성공",
                    }
                    results.append(trade_result)
                    successful_trades.append(trade_result)
                else:
                    results.append({
                        "날짜": current.time,
                        "매수/매도": "매수신호",
                        "특징": "상승 잉태형",
                        "매매시점": "미충족",
                        "손절조건": "미충족" if not stoploss else "충족",
                        "매수 성공 여부": "실패",
                    })
            else:
                results.append({
                    "날짜": current.time,
                    "매수/매도": "조건 미충족",
                    "특징": "미충족",
                    "매매시점": "미충족",
                    "손절조건": "미충족",
                    "매수 성공 여부": "실패",
                })

        return results, successful_trades
    
            # 매수/매도 신호 판단
signals, successful_trades = harami_logic(ohlc)

# 전체 신호 출력
print("\n전체 신호:")
for signal in signals:
    print(signal)

# 매수 성공 목록 출력
print("\n매수 성공 목록:")
for trade in successful_trades:
    print(trade)

In [17]:
def doji_star_logic(ohlc):
        """
        상승 도지 스타 매매 로직.
        :return: 매수 성공 목록과 개별 신호
        """
        results = []
        successful_trades = []

        if len(ohlc) < 3:
            raise ValueError("3일 이상의 데이터가 필요합니다.")

        for i in range(2, len(ohlc)):
            d_2 = ohlc[i - 2]  # D-2 데이터
            d_1 = ohlc[i - 1]  # D-1 데이터
            current = ohlc[i]  # 당일 데이터

            # D-2 조건: D-2 종가 < D-2 시초가 (음봉)
            d_2_condition = d_2.close < d_2.open
            print(f"\n[D-2 조건] 날짜: {d_2.time}")
            print(f"  D-2 종가 < D-2 시초가: {d_2.close} < {d_2.open} = {d_2_condition}")

            # D-1 조건
            d_1_condition = (
                d_1.close == d_1.open and  # 도지 조건
                d_1.open < d_2.low         # D-1 시초가 < D-2 최저가
            )
            print(f"\n[D-1 조건] 날짜: {d_1.time}")
            print(f"  D-1 종가 = D-1 시초가: {d_1.close} == {d_1.open} = {d_1.close == d_1.open}")
            print(f"  D-1 시초가 < D-2 최저가: {d_1.open} < {d_2.low} = {d_1.open < d_2.low}")

            # 매매 시점: 당일 종가 > D-2 최고가
            buy_signal = current.close > d_2.high
            print(f"\n[매매 시점] 날짜: {current.time}")
            print(f"  당일 종가 > D-2 최고가: {current.close} > {d_2.high} = {buy_signal}")

            # 손절 조건: 당일 종가 < D-1 최저가
            stoploss = current.close < d_1.low
            print(f"\n[손절 조건] 날짜: {current.time}")
            print(f"  당일 종가 < D-1 최저가: {current.close} < {d_1.low} = {stoploss}")

            # 모든 조건 충족 여부 확인
            all_conditions_met = d_2_condition and d_1_condition
            print(f"\n[전체 조건 충족 여부] 날짜: {current.time}")
            print(f"  D-2 조건: {d_2_condition}")
            print(f"  D-1 조건: {d_1_condition}")
            print(f"  모든 조건 충족: {all_conditions_met}")

            if all_conditions_met:
                if buy_signal:
                    trade_result = {
                        "날짜": current.time,
                        "매수/매도": "매수신호",
                        "특징": "상승 도지 스타",
                        "매매시점": "P > h",
                        "손절조건": "미충족" if not stoploss else "충족",
                        "매수 성공 여부": "성공",
                    }
                    results.append(trade_result)
                    successful_trades.append(trade_result)
                else:
                    results.append({
                        "날짜": current.time,
                        "매수/매도": "매수신호",
                        "특징": "상승 도지 스타",
                        "매매시점": "미충족",
                        "손절조건": "미충족" if not stoploss else "충족",
                        "매수 성공 여부": "실패",
                    })
            else:
                results.append({
                    "날짜": current.time,
                    "매수/매도": "조건 미충족",
                    "특징": "미충족",
                    "매매시점": "미충족",
                    "손절조건": "미충족",
                    "매수 성공 여부": "실패",
                })

        return results, successful_trades
    
            # 매수/매도 신호 판단
signals, successful_trades = doji_star_logic(ohlc)

# 전체 신호 출력
print("\n전체 신호:")
for signal in signals:
    print(signal)

# 매수 성공 목록 출력
print("\n매수 성공 목록:")
for trade in successful_trades:
    print(trade)    


[D-2 조건] 날짜: 2020-06-01 00:00:00+09:00
  D-2 종가 < D-2 시초가: 1.505E+4 < 1.465E+4 = False

[D-1 조건] 날짜: 2020-06-02 00:00:00+09:00
  D-1 종가 = D-1 시초가: 1.54E+4 == 1.485E+4 = False
  D-1 시초가 < D-2 최저가: 1.485E+4 < 1.405E+4 = False

[매매 시점] 날짜: 2020-06-03 00:00:00+09:00
  당일 종가 > D-2 최고가: 1.515E+4 > 1.55E+4 = False

[손절 조건] 날짜: 2020-06-03 00:00:00+09:00
  당일 종가 < D-1 최저가: 1.515E+4 < 1.455E+4 = False

[전체 조건 충족 여부] 날짜: 2020-06-03 00:00:00+09:00
  D-2 조건: False
  D-1 조건: False
  모든 조건 충족: False

[D-2 조건] 날짜: 2020-06-02 00:00:00+09:00
  D-2 종가 < D-2 시초가: 1.54E+4 < 1.485E+4 = False

[D-1 조건] 날짜: 2020-06-03 00:00:00+09:00
  D-1 종가 = D-1 시초가: 1.515E+4 == 1.57E+4 = False
  D-1 시초가 < D-2 최저가: 1.57E+4 < 1.455E+4 = False

[매매 시점] 날짜: 2020-06-04 00:00:00+09:00
  당일 종가 > D-2 최고가: 1.395E+4 > 1.545E+4 = False

[손절 조건] 날짜: 2020-06-04 00:00:00+09:00
  당일 종가 < D-1 최저가: 1.395E+4 < 1.505E+4 = True

[전체 조건 충족 여부] 날짜: 2020-06-04 00:00:00+09:00
  D-2 조건: False
  D-1 조건: False
  모든 조건 충족: False

[D-2 조건] 날짜: 2020-0

In [14]:
def simulate_trading_with_morning_star(ohlc_data, trade_value):
    """
    샛별형 매매 시뮬레이션 (실현 손익 계산 제외).
    :param ohlc_data: OHLC 데이터 (KisDomesticDailyChartBar 객체 리스트)
    :param trade_value: 매매에 사용할 금액 (KRW)
    :return: 매매 기록 및 매수/매도 신호
    """
    # 초기화
    trading_history = {
        'history': [],  # 매매 기록
    }

    # 시뮬레이션 데이터 저장
    buy_signals = []
    sell_signals = []
    timestamps = []

    # 시뮬레이션 실행
    for i in range(2, len(ohlc_data)):
        d_2 = ohlc_data[i - 2]
        d_1 = ohlc_data[i - 1]
        current = ohlc_data[i]

        # 샛별형 매수 조건
        d_2_condition = d_2.close < d_2.open  # D-2 음봉
        d_2_long_bear = abs(d_2.close - d_2.open) >= (float(d_2.open) * 0.02)  # 장대음봉

        d_1_condition = (
            d_2.close > d_1.close > d_1.open  # D-2 종가 > D-1 종가 > D-1 시초가
        )

        # 매매 시점 조건
        buy_signal_low = current.low > d_1.close  # LL(오늘 최저가) > E(D-1 종가)
        buy_signal_high = current.close > d_2.high  # P > h

        all_conditions_met = d_2_condition and d_2_long_bear and d_1_condition

        if all_conditions_met and (buy_signal_low or buy_signal_high):
            # 매수
            buy_price = float(current.open)  # 매수가는 당일 시초가로 설정
            stop_loss = float(d_2.low)  # stop loss는 D-2 최저가
            quantity = math.floor(trade_value / buy_price)
            if quantity > 0:
                trading_history['history'].append({
                    'position': 'BUY',
                    'price': buy_price,
                    'quantity': quantity,
                    'stop_loss': stop_loss,
                })

                # 매수 신호 기록
                buy_signals.append((current.time, buy_price))
                timestamps.append(current.time)
                print(f"샛별형 매수 시점: {current.time}, 매수가: {buy_price}, 매수량: {quantity}, 손절가: {stop_loss}")

        # 매도 조건 확인 및 실행
        for trade in trading_history['history']:
            if trade['position'] == 'BUY' and trade['quantity'] > 0:  # 매수 수량이 있어야 매도 가능
                target_price = trade['price'] + 2 * (trade['price'] - trade['stop_loss'])
                if current.close <= trade['stop_loss']:
                    # 손절 매도
                    sell_quantity = trade['quantity']  # 매도 수량은 남은 매수 수량만큼
                    sell_price = float(trade['stop_loss'])
                    trading_history['history'].append({
                        'position': 'SELL',
                        'price': sell_price,
                        'quantity': sell_quantity,
                    })
                    sell_signals.append((current.time, sell_price))
                    print(f"손절 매도: {current.time}, 매도가: {sell_price}, 매도량: {sell_quantity}")
                    trade['quantity'] = 0  # 남은 수량 감소
                elif current.close >= target_price:
                    # 목표 수익 매도
                    sell_quantity = trade['quantity']  # 매도 수량은 남은 매수 수량만큼
                    sell_price = float(target_price)
                    trading_history['history'].append({
                        'position': 'SELL',
                        'price': sell_price,
                        'quantity': sell_quantity,
                    })
                    sell_signals.append((current.time, sell_price))
                    print(f"목표 매도: {current.time}, 매도가: {sell_price}, 매도량: {sell_quantity}")
                    trade['quantity'] = 0  # 남은 수량 감소

    return trading_history, buy_signals, sell_signals, timestamps

trade_value = 1000000
            # 매수/매도 신호 판단
trading_history, buy_signals, sell_signals, timestamps = simulate_trading_with_morning_star(ohlc,trade_value)



# 최종 결과 출력
print("\n[최종 매매 기록]")
for trade in trading_history['history']:
    print(trade)

샛별형 매수 시점: 2023-10-27 00:00:00+09:00, 매수가: 425500.0, 매수량: 2, 손절가: 443000.0
목표 매도: 2023-10-27 00:00:00+09:00, 매도가: 390500.0, 매도량: 2

[최종 매매 기록]
{'position': 'BUY', 'price': 425500.0, 'quantity': 0, 'stop_loss': 443000.0}
{'position': 'SELL', 'price': 390500.0, 'quantity': 2}
